<a href="https://colab.research.google.com/github/raaraya1/Personal-Proyects/blob/main/Proyectos/Mapas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Modulo Mapas**

## **Conociendo las bibliotecas necesarias**

Ahora ultimo he estado buscando metodos para visualizar las soluciones arrojadas en los modelos de transporte que he estado desarrollando y en consecuencia me encontre dos bibliotecas que me podrian ayudar con proceso.

Estas son:

- folium (biblioteca principal)
- openrouteservice (biblioteca secundaria)

**Nota1**: Si se esta trabajando desde Google Colab, primero tendremos que descargar la libreria de openrouteservice con el comando `!pip install openrouteservice`.

**Nota2**: Adicionalmente, para utilizar la biblioteca de openrouteservice se requiere crearse una cuenta en https://openrouteservice.org, luego en la seccion Dashboard, hay que crearse un token (gratis) y asi luego obtenemos la clave.

In [ ]:
!pip install openrouteservice
import openrouteservice
import folium

In [ ]:
# Llamar al cliente
client=openrouteservice.Client(key='Colocar la clave aqui') 


In [1]:
# Direcciones de prueba ()
coordenadas = [[-72.99018144607545,-41.309445802681246], 
              [-72.99014925956727,-41.30772114911309], 
              [-72.99514889717103,-41.31167811362131], 
              [-72.98510670661928,-41.311670054874526], 
              [-72.98683404922487,-41.310420937078746], 
              [-72.98252105712892,-41.3063833023091]]


In [ ]:
# Matrix de distancias
matrix_de_distancias = client.distance_matrix(locations=coordenadas, metrics = ("duration", "distance"), units = "km")
matrix_de_distancias['distances']

[[0.0, 0.83, 0.61, 1.17, 0.7, 1.51],
 [0.71, 0.0, 0.98, 1.54, 1.08, 1.89],
 [0.62, 1.13, 0.0, 1.21, 0.85, 1.66],
 [1.17, 1.68, 1.21, 0.0, 0.32, 0.91],
 [0.71, 1.22, 0.84, 0.32, 0.0, 0.86],
 [1.52, 2.03, 1.65, 0.9, 0.86, 0.0]]

In [ ]:
import numpy as np

In [ ]:
# Voy a hacer una lista nueva para la libreria de folium (necesita las coordenadas
# en formato [lat, lon])
coordenadas_folium = [[i[1], i[0]] for i in coordenadas]

# Para hacer el mapa (voy a utilizar la primera locacion como referencia)
punto_central = np.mean(coordenadas_folium, axis=0)
map = folium.Map(location=punto_central, zoom_start=15)

# Colocamos el resto de los lugares
for indx, coord in enumerate(coordenadas_folium):
  folium.Marker(coord, popup=f'P{indx+1}').add_to(map)

map

In [ ]:
# Creamos las rutas
## Ojo que no calcula rutas por sobre 6000 km
rutas = []
for i in coordenadas:
  for j in coordenadas:
    if i != j:
      input = [i, j]
      rutas.append(client.directions(coordinates=input, profile='driving-car', format='geojson'))


In [ ]:
dic_routes = {}
for i in range(len(rutas)):
  dic_routes[f'ruta_{i+1}'] = rutas[i]


In [ ]:
for i in dic_routes:
  folium.GeoJson(dic_routes[i], name=i).add_to(map)

folium.LayerControl().add_to(map)
map

## **Creando el modulo**

In [13]:
!pip install openrouteservice
import openrouteservice
import folium
import numpy as np

In [32]:
class Mapas():
  def __init__(self, key, lugares):
    self.key = key
    self.client = openrouteservice.Client(key=key)
    self.matrix_de_distancias = [] 
    self.lugares = lugares
    

  def matriz_de_distancias(self, unidad='km'):
    matrix_de_distancias = self.client.distance_matrix(locations=self.lugares, metrics = ("duration", "distance"), units = unidad)
    return matrix_de_distancias['distances']

  def Mapa_con_lugares(self):
    coordenadas_folium = [[i[1], i[0]] for i in self.lugares]
    punto_central = np.mean(coordenadas_folium, axis=0)
    map = folium.Map(location=punto_central, zoom_start=15)

    for indx, coord in enumerate(coordenadas_folium):
      folium.Marker(coord, popup=f'P{indx+1}').add_to(map)

    return map

  def Mapa_con_rutas(self, rutas_especificas=0):
    map = self.Mapa_con_lugares()

    if rutas_especificas == 0:
      rutas = []
      for i in self.lugares:
        for j in self.lugares:
          if i != j:
            input = [i, j]
            rutas.append(self.client.directions(coordinates=input, profile='driving-car', format='geojson'))

      dic_routes = {}
      for i in range(len(rutas)):
        dic_routes[f'ruta_{i+1}'] = rutas[i]

      for i in dic_routes:
        folium.GeoJson(dic_routes[i], name=i).add_to(map)

      folium.LayerControl().add_to(map)

    elif rutas_especificas != 0:
      dic_routes = {}
      for i in rutas_especificas:
        input = [rutas_especificas[i][0], rutas_especificas[i][1]]
        ruta = self.client.directions(coordinates=input, profile='driving-car', format='geojson')
        dic_routes[i] = ruta
      
      for i in dic_routes:
        folium.GeoJson(dic_routes[i], name=i).add_to(map)

      folium.LayerControl().add_to(map)

    return map


In [34]:
# Datos de prueba

key='Colocar la clave aqui'

coordenadas = [[-72.99018144607545,-41.309445802681246], 
              [-72.99014925956727,-41.30772114911309], 
              [-72.99514889717103,-41.31167811362131], 
              [-72.98510670661928,-41.311670054874526], 
              [-72.98683404922487,-41.310420937078746], 
              [-72.98252105712892,-41.3063833023091]]

rutas_especificas = {'ruta1': [[-72.99018144607545,-41.309445802681246], [-72.99014925956727,-41.30772114911309]],
                     'ruta2': [[-72.98510670661928,-41.311670054874526], [-72.98252105712892,-41.3063833023091]]}

In [35]:
# Ahora probemos como quedo este modulo
mapa = Mapas(key=key, lugares=coordenadas)

In [28]:
# matriz de distancias
mapa.matriz_de_distancias()

[[0.0, 0.83, 0.61, 1.17, 0.7, 1.51],
 [0.71, 0.0, 0.98, 1.54, 1.08, 1.89],
 [0.62, 1.13, 0.0, 1.21, 0.85, 1.66],
 [1.17, 1.68, 1.21, 0.0, 0.32, 0.91],
 [0.71, 1.22, 0.84, 0.32, 0.0, 0.86],
 [1.52, 2.03, 1.65, 0.9, 0.86, 0.0]]

In [29]:
# mapa con los lugares
mapa.Mapa_con_lugares()

In [36]:
# mapa por defecto con todas las rutas
mapa.Mapa_con_rutas()

In [37]:
# Mapa con rutas especificas
mapa.Mapa_con_rutas(rutas_especificas=rutas_especificas)